# Pull parser with XML output: identity transformation

## Check the input XML

In [1]:
with open('flattened.xml') as input:
    print(input.read())

<?xml version="1.0" encoding="UTF-8"?>
<root xmlns:th="http://www.blackmesatech.com/2017/nss/trojan-horse">
    <p th:sID="d1e3"/>This is a <word th:sID="d1e5"/>paragraph<word th:eID="d1e5"/> that contains
    some <nonTrojan type="test"/> stuff.<p th:eID="d1e3"/>
    <p th:sID="d1e9"/>This is <emphasis role="bold">another</emphasis> paragraph <phrase
        th:sID="d1e11"/><word th:sID="d1e12"/>that<word th:eID="d1e12"/>
    <word th:sID="d1e15"/>contains<word th:eID="d1e15"/>
    <word th:sID="d1e18"/>more<word th:eID="d1e18"/><phrase th:eID="d1e11"/> stuff.<p th:eID="d1e9"
    />
</root>



## Transform it

In [2]:
from xml.dom.pulldom import CHARACTERS, START_ELEMENT, parseString, END_ELEMENT
from xml.dom.minidom import Document


class Stack(list):
    def push(self, item):
        self.append(item)

    def peek(self):
        return self[-1]

open_elements = Stack()
d = Document()
open_elements.push(d)
output = []

with open('flattened.xml') as input:
    for event, node in parseString(input.read()):
        if event == START_ELEMENT:
            open_elements.peek().appendChild(node)
            open_elements.push(node)
        if event == END_ELEMENT: 
            open_elements.pop()
        elif event == CHARACTERS:
            t = d.createTextNode(node.data)
            open_elements.peek().appendChild(t)

print(d.toxml())

<?xml version="1.0" ?><root xmlns:th="http://www.blackmesatech.com/2017/nss/trojan-horse">
    <p th:sID="d1e3"/>This is a <word th:sID="d1e5"/>paragraph<word th:eID="d1e5"/> that contains
    some <nonTrojan type="test"/> stuff.<p th:eID="d1e3"/>
    <p th:sID="d1e9"/>This is <emphasis role="bold">another</emphasis> paragraph <phrase th:sID="d1e11"/><word th:sID="d1e12"/>that<word th:eID="d1e12"/>
    <word th:sID="d1e15"/>contains<word th:eID="d1e15"/>
    <word th:sID="d1e18"/>more<word th:eID="d1e18"/><phrase th:eID="d1e11"/> stuff.<p th:eID="d1e9"/>
</root>
